In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf

from typing import Union, List, Tuple
from numpy.typing import ArrayLike, NDArray
from loguru import logger
from scipy.fft import irfft, rfftfreq
from IPython.display import Audio
from pathlib import Path

from pyFDN.fdn import FDN
from pyFDN.feedback_matrix import FeedbackMatrix, FeedbackMatrixType
from pyFDN.delay_line import generate_coprime_delay_line_lengths
from pyFDN.utils import ms_to_samps, estimate_echo_density

seed = 482452
np.random.seed(seed)

#### FDN parameters

In [ ]:
# sampling rate
fs = 48000
# number of delay lines
N = 8
frame_size = 2**8

# we want a binaural output
num_input = 1
num_output = 2

# input gains
b = np.random.randn(N, num_input)
# change coefficients of c so that the columns are orthonormal
c = np.random.randn(num_output, N)
direct_gain = 0.5 * np.ones((num_output, num_input))

# delay lengths should be co-prime
# constrict delay range to be between 50 and 100ms
delay_range_ms = np.array([10, 20])
delay_lengths = generate_coprime_delay_line_lengths(delay_range_ms, N, fs)
logger.info(f'The delay line lengths are {delay_lengths} samples')

# how long should the impulse response be
ir_len = ms_to_samps(300, fs)
# create an impulse
input_data = np.zeros((num_input, ir_len))
input_data[:, 0] = 1.0


#### Desired frequency based T60

In [ ]:
t60_freqs = np.array([63, 125, 250, 500, 1000, 2000, 4000, 8000, 16000])
des_t60_ms = 1e3 * np.array([0.4, 0.4, 0.56, 0.3, 0.35, 0.2, 0.25, 0.23, 0.24])
iir_filter_order = 10
warp_factor = -0.77
num_freq_bins = 2**10

#### Initialise FDN

In [ ]:
fdn = FDN(fs, num_input, num_output, N, frame_size, seed)

fdn.init_io_gains(b, c)
fdn.init_direct_gain(direct_gain)
fdn.init_delay_line_lengths(delay_lengths)
fdn.init_feedback_matrix()
fdn.init_absorption_gains(des_t60_ms, t60_freqs, iir_filter_order, warp_factor)
fdn.init_delay_lines()


#### Plot the desired and achieved T60 filters in each delay line

In [ ]:
fdn.absorption.plot_t60_filter_response()

#### Plot the FDN's impulse response

In [ ]:
fdn_ir = fdn.process(input_data)

time_vector = np.arange(0, ir_len/fs, 1.0/fs)
plt.figure()
plt.plot(time_vector, fdn_ir.T)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title(f'FDN IR with velvet feedback matrix and T60 of {des_t60_ms} ms')

del fdn